In [229]:
import tweepy
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from langdetect import detect

 
# import twitter_credentials
import numpy as np
import pandas as pd
consumer_key = "cFf7im7BH68xO9qh3zEsv3nFz"
consumer_secret = "2QyodEVK63XYE5D9RFPAo0I53rhBOsNocQGpGB8rapmEqxDnJi"
access_token = "1265319352795975680-EJBIU55ZHZnjn8svR420cqVuU9evRL"
access_token_secret = "i8h2utF6b2l2Uh7Vpg6c2mnPOO1CuthGdbyWtHSWdIURP"



# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets
    
# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        return auth

# # # # TWITTER STREAMER # # # #

# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)


class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])
        df['Date Created'] = np.array([tweet.created_at for tweet in tweets])
        df["24H"] = df["Date Created"].apply(lambda x: 0 if (datetime.utcnow()-x)>timedelta(1) else 1)
        df.drop(columns=["Date Created"],axis=1,inplace=True)
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df.drop(df[df['likes']<=1000].index,inplace=True)

        return df
class FilterData():
    
     def filter_RT():
        df["Retweets"] = df["Tweets"].apply(lambda x: True if x[:2]=="RT" else False)
        df.drop(df[df["Retweets"]==True].index,inplace=True)
        df.drop(columns=["Retweets"],axis=1,inplace=True)
        df.index = [i for i in range(df["Tweets"].count())]
     def filter_lang():
        df["Lang"] = df["Tweets"].apply(lambda x: False if detect(x)=="en" else True)
        df.drop(df[df["Lang"]==True].index,inplace=True)
        df.drop(columns=["Lang"],axis=1,inplace=True)
        df.index = [i for i in range(df["Tweets"].count())]

        
if __name__ == '__main__':

    twitter_client = TwitterClient()
    
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = api.user_timeline(screen_name="narendramodi", count=200)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    FilterData.filter_RT()
    FilterData.filter_lang()
    df.to_csv('modi-tweets.csv')
df

,Tweets,24H,likes
0,India and the Philippines will cooperate to re...,1,16656
1,Had a useful exchange with President Rodrigo D...,1,32780
2,Shri Ved Marwah Ji will be remembered for his ...,0,61272
3,"Rwanda is, and will remain, an important pilla...",0,22369
4,"Thank you, my friend President @PaulKagame. \n...",0,36925
...,...,...,...
161,Excellent effort by @CNNnews18. They have wond...,0,28441
162,Discussed COVID-19 pandemic with good friend P...,0,53268
163,Please thank Swetha for this creative and affe...,0,76360
164,Good gesture by the @DelhiPolice.\n\nOur Gurud...,0,67072


In [230]:
urls=[]
def filter_url(x):
        if "https://" in x:
            index = x.find("https://")
            urls.append(x[index:])
        else:
            urls.append("0")
df["Tweets"].apply(filter_url)
df["URL"] = urls

In [231]:
df

,Tweets,24H,likes,URL
0,India and the Philippines will cooperate to re...,1,16656,https://t.co/WDn16d4Biz
1,Had a useful exchange with President Rodrigo D...,1,32780,https://t.co/sAkeSHDCKE
2,Shri Ved Marwah Ji will be remembered for his ...,0,61272,https://t.co/uUyZ58iBIy
3,"Rwanda is, and will remain, an important pilla...",0,22369,0
4,"Thank you, my friend President @PaulKagame. \n...",0,36925,https://t.co/9jrdBpmLcC
...,...,...,...,...
161,Excellent effort by @CNNnews18. They have wond...,0,28441,https://t.co/VqUJZZ78lM
162,Discussed COVID-19 pandemic with good friend P...,0,53268,https://t.co/zJhncRcJOy
163,Please thank Swetha for this creative and affe...,0,76360,https://t.co/XqtliBFlgW
164,Good gesture by the @DelhiPolice.\n\nOur Gurud...,0,67072,https://t.co/qPRkYD2Knt


In [232]:
from selenium import webdriver
def tweet_extraction(x):
    driver=webdriver.Chrome(executable_path='C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe') #the path where chromedriver.exe is installed in your local system
    driver.get(x)
    ele=driver.find_elements_by_class_name('css-1dbjc4n')
    for r in ele:
        print(r.text.strip())
    driver.close()

# RUN THE BELOW CELL IF YOU ARE CRAZY

In [217]:
for i in s:
    if i!="0":
        tweet_extraction(i[:i.find(" ")])
#         '"'+i[:i.find(" ")]+'"'